# Interactive Roary Gene Presence–Absence Heatmap (Python + Plotly)

This Jupyter notebook builds an **interactive heatmap** from a Roary `gene_presence_absence*.csv` file.

It will:
- Load the Roary presence–absence table
- Randomly select **20 genomes**
- Select up to **20 genes** with **≥ 50% prevalence**
- Build a **0/1 presence–absence matrix**
- Color each **genome with a unique color** (white = absence)
- Display an **interactive Plotly heatmap with hover tooltips**

> **Before running:** make sure your file `gene_presence_absence_reps.csv` is available in the `data/` folder next to this notebook (or update the path below).


In [ ]:
import os
import random

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

In [ ]:
# === User parameters ===

# Path to your Roary gene presence–absence file
data_path = os.path.join("data", "gene_presence_absence_reps.csv")

# Number of metadata columns before genome columns (Roary default is 14)
meta_cols = 14

# How many genomes and genes to sample
n_genomes_target = 20
n_genes_target = 20

# Minimum prevalence across selected genomes (e.g. 0.50 = 50%)
prevalence_threshold = 0.50

# Random seed for reproducibility
random_seed = 123

In [ ]:
# === Load Roary gene_presence_absence file ===

if not os.path.exists(data_path):
    raise FileNotFoundError(f"Roary file not found at: {data_path}")

print(f"Reading: {data_path}")
gpa = pd.read_csv(data_path)

print(f"Data shape: {gpa.shape[0]} genes x {gpa.shape[1]} columns")

In [ ]:
# === Identify genome columns and sample 20 genomes ===

all_cols = gpa.columns.tolist()
genome_cols = all_cols[meta_cols:]

print(f"Total genomes detected: {len(genome_cols)}")


if len(genome_cols) < n_genomes_target:
    raise ValueError(
        f"Requested {n_genomes_target} genomes but only {len(genome_cols)} available."
    )

random.seed(random_seed)
selected_genomes = random.sample(genome_cols, n_genomes_target)
print("\nSelected genomes (20):")
for g in selected_genomes:
    print(" -", g)

In [ ]:
# === Build 0/1 presence–absence matrix ===

sub = gpa[["Gene"] + selected_genomes].copy()

# Convert to 0/1: non-empty = 1, NA/empty = 0
for col in selected_genomes:
    sub[col] = sub[col].fillna("").astype(str)
    sub[col] = np.where(sub[col] == "", 0, 1)

# Set Gene as index
pa_mat = sub.set_index("Gene").astype(int)
print("\nPresence–absence matrix shape (genes x genomes):", pa_mat.shape)

In [ ]:
# === Filter genes by prevalence (≥ threshold) and sample up to 20 genes ===

prevalence = pa_mat.mean(axis=1)
mask = prevalence >= prevalence_threshold

high_prev = pa_mat.loc[mask]

print(f"Genes with ≥ {prevalence_threshold*100:.0f}% prevalence: {high_prev.shape[0]}")


if high_prev.shape[0] == 0:
    raise ValueError("No genes meet the prevalence threshold. Try lowering it.")

if high_prev.shape[0] > n_genes_target:
    high_prev = high_prev.sample(n_genes_target, random_state=random_seed)

print("Final matrix shape (genes x genomes):", high_prev.shape)

# For convenience
gene_names = high_prev.index.tolist()
genome_names = high_prev.columns.tolist()

high_prev

In [ ]:
# === Build color index matrix and hover text ===

# color_mat: 0 = absent, j = genome-specific color index
n_genomes = len(genome_names)
n_genes = len(gene_names)

color_mat = np.zeros_like(high_prev.values, dtype=int)

for j in range(n_genomes):
    color_mat[:, j] = np.where(high_prev.iloc[:, j] == 1, j + 1, 0)

# Prepare hover text matrix
hover_text = []
for i, gene in enumerate(gene_names):
    row = []
    for j, genome in enumerate(genome_names):
        present = high_prev.iloc[i, j] == 1
        row.append(
            f"Gene: {gene}<br>Genome: {genome}<br>Presence: {'Present' if present else 'Absent'}"
        )
    hover_text.append(row)

color_mat

In [ ]:
# === Build custom colorscale: 0 = white, 1..N = distinct colors per genome ===

# Use a qualitative palette and repeat if needed
base_palette = px.colors.qualitative.Set3
repeats = int(np.ceil((n_genomes) / len(base_palette)))
palette = (base_palette * repeats)[:n_genomes]

# colors_for_values[0] = white (absence), 1..n_genomes = palette
colors_for_values = ['white'] + palette

colorscale = []
max_index = n_genomes
for idx, color in enumerate(colors_for_values):
    # normalize idx to [0,1]
    val = idx / max_index if max_index > 0 else 0
    colorscale.append([val, color])

colorscale

In [ ]:
# === Plot interactive heatmap ===

fig = go.Figure(
    data=go.Heatmap(
        z=color_mat,
        x=genome_names,
        y=gene_names,
        colorscale=colorscale,
        showscale=False,
        text=hover_text,
        hoverinfo="text"
    )
)

fig.update_layout(
    title=f"Roary gene presence–absence (subset: {n_genes} genes × {n_genomes} genomes)",
    xaxis_title="Genomes",
    yaxis_title="Genes",
    xaxis=dict(tickangle=0),
    yaxis=dict(tickangle=0),
    autosize=True,
)

fig.show()

In [ ]:
# === (Optional) Save as a standalone HTML file ===

output_dir = "figures"
os.makedirs(output_dir, exist_ok=True)

html_path = os.path.join(output_dir, "pangenome_heatmap_20x20_python.html")
fig.write_html(html_path)

print(f"Interactive HTML saved to: {html_path}")